# test note


* jupyterはコンテナ起動すること
* テストベッド一式起動済みであること


In [1]:
!pip install --upgrade pip
!pip install --force-reinstall ../lib/ait_sdk-0.1.5-py3-none-any.whl

     |████████████████████████████████| 1.5 MB 6.4 MB/s eta 0:00:01     |██████████▉                     | 522 kB 6.4 MB/s eta 0:00:01     |██████████████████████▍         | 1.1 MB 6.4 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.3.1
    Uninstalling pip-20.3.1:
      Successfully uninstalled pip-20.3.1
Processing /workdir/root/lib/ait_sdk-0.1.5-py3-none-any.whl
     |████████████████████████████████| 14.9 MB 11.3 MB/s eta 0:00:01   |██████▏                         | 2.9 MB 6.3 MB/s eta 0:00:02     |████████████▎                   | 5.7 MB 6.3 MB/s eta 0:00:02     |██████████████████████████▍     | 12.3 MB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 465 kB 10.9 MB/s eta 0:00:01
  Using cached nbformat-5.0.8-py3-none-any.whl (172 kB)
  Using cached nbconvert-6.0.7-py3-none-any.whl (552 kB)
     |████████████████████████████████| 95 kB 3.6 MB/s  eta 0:00:01
     |████████████████████████████████| 25.9 MB 14.4 MB/s eta 0:00:01   |█

In [2]:
from pathlib import Path
import pprint
from ait_sdk.test.hepler import Helper
import json

In [3]:
# settings cell

# mounted dir
root_dir = Path('/workdir/root/ait')

ait_name='eval_regression_analyze_rmse_and_mae_tf2.3'
ait_version='0.1'

ait_full_name=f'{ait_name}_{ait_version}'
ait_dir = root_dir / ait_full_name

td_name=f'{ait_name}_test'

# (dockerホスト側の)インベントリ登録用アセット格納ルートフォルダ
current_dir = %pwd
with open(f'{current_dir}/config.json', encoding='utf-8') as f:
    json_ = json.load(f)
    root_dir = json_['host_ait_root_dir']
    is_container = json_['is_container']
invenotory_root_dir = f'{root_dir}\\ait\\{ait_full_name}\\local_qai\\inventory'

# entry point address
# コンテナ起動かどうかでポート番号が変わるため、切り替える
if is_container:
    backend_entry_point = 'http://host.docker.internal:8888/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:8888/qai-ip/api/0.0.1'
else:
    backend_entry_point = 'http://host.docker.internal:5000/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:6000/qai-ip/api/0.0.1'

# aitのデプロイフラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_ait = True

# インベントリの登録フラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_inventory = True


In [4]:
helper = Helper(backend_entry_point=backend_entry_point, 
                ip_entry_point=ip_entry_point,
                ait_dir=ait_dir,
                ait_full_name=ait_full_name)

In [5]:
# health check

helper.get_bk('/health-check')
helper.get_ip('/health-check')

<Response [200]>
{'Code': 0, 'Message': 'alive.'}
<Response [200]>
{'Code': 0, 'Message': 'alive.'}


<Response [200]>

In [6]:
# create ml-component
res = helper.post_ml_component(name=f'MLComponent_{ait_full_name}', description=f'Description of {ait_full_name}', problem_domain=f'ProbremDomain of {ait_full_name}')
helper.set_ml_component_id(res['MLComponentId'])

<Response [200]>
{'MLComponentId': 16,
 'Result': {'Code': 'P22000', 'Message': 'add ml-component success.'}}


In [7]:
# deploy AIT
if is_init_ait:
    helper.deploy_ait_non_build()
else:
    print('skip deploy AIT')

<Response [400]>
{'Code': 'T54000',
 'Message': 'already exist ait = '
            'eval_regression_analyze_rmse_and_mae_tf2.3-0.1'}
<Response [200]>
{'Code': 'D00001', 'Message': 'Deploy success'}


In [8]:
res = helper.get_data_types()
model_data_type_id = [d for d in res['DataTypes'] if d['Name'] == 'model'][0]['Id']
dataset_data_type_id = [d for d in res['DataTypes'] if d['Name'] == 'dataset'][0]['Id']
res = helper.get_file_systems()
unix_file_system_id = [f for f in res['FileSystems'] if f['Name'] == 'UNIX_FILE_SYSTEM'][0]['Id']
windows_file_system_id = [f for f in res['FileSystems'] if f['Name'] == 'WINDOWS_FILE'][0]['Id']

In [9]:
# add inventories

if is_init_inventory:
    inv1_name = helper.post_inventory('dataset_for_verification', dataset_data_type_id, windows_file_system_id, 
                                      f'{invenotory_root_dir}\\dataset_for_verification\\test_dataset.csv',
                                      '幸福度調査データで使用した学習用データ', ['csv'])
    inv2_name = helper.post_inventory('trained_model', model_data_type_id, windows_file_system_id, 
                                      f'{invenotory_root_dir}\\trained_model\\model_2.h5',
                                      '幸福度調査データで構築した幸福度予測モデル', ['h5'])
else:
    print('skip add inventories')

<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}


In [10]:
# get ait_json and inventory_jsons

res_json = helper.get_bk('/QualityMeasurements/RelationalOperators', is_print_json=False).json()
eq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '=='][0])
nq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '!='][0])
gt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>'][0])
ge_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>='][0])
lt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<'][0])
le_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<='][0])

res_json = helper.get_bk('/testRunners', is_print_json=False).json()
ait_json = [j for j in res_json['TestRunners'] if j['Name'] == ait_name][-1]

inv_1_json = helper.get_inventory(inv1_name)
inv_2_json = helper.get_inventory(inv2_name)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [11]:
# add teast_descriptions

helper.post_td(td_name, ait_json['QualityDimensionId'],
               quality_measurements=[
                   {"Id":ait_json['Report']['Measures'][0]['Id'], "Value":"0.5", "RelationalOperatorId":lt_id, "Enable":True},
                   {"Id":ait_json['Report']['Measures'][1]['Id'], "Value":"0.5", "RelationalOperatorId":lt_id, "Enable":True}
               ],
               target_inventories=[
                   {"Id":1, "InventoryId": inv_2_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][0]['Id']},
                   {"Id":2, "InventoryId": inv_1_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][1]['Id']}
               ], 
               test_runner={
                "Id":ait_json['Id'],
                "Params":[
                   {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][0]['Id'], "Value":"Happiness Score"}
               ]
               })

<Response [200]>
{'Result': {'Code': 'T22000', 'Message': 'append test description success.'}}


In [12]:
# get test_description_jsons
td_1_json = helper.get_td(td_name)

<Response [200]>


In [13]:
# run test_descriptions
helper.post_run_and_wait(td_1_json['Id'])

<Response [200]>
{'Job': {'Id': '16', 'StartDateTime': '2021-01-28 14:33:09.491451+09:00'},
 'Result': {'Code': 'R12000', 'Message': 'job launch success.'}}
[{'Id': 16,
  'Result': 'NG',
  'ResultDetail': 'RMSE : NG.\nMAE : NG.\n',
  'Status': 'DONE',
  'TestDescriptionID': 16}]


In [14]:
res_json = helper.get_td_detail(td_1_json['Id'])
pprint.pprint(res_json)

<Response [200]>
{'Result': {'Code': 'T32000', 'Message': 'get detail success.'},
 'TestDescriptionDetail': {'Id': 16,
                           'Name': 'eval_regression_analyze_rmse_and_mae_tf2.3_test',
                           'Opinion': '',
                           'QualityDimension': {'Id': 5,
                                                'Name': 'Accuracy_of_trained_model'},
                           'QualityMeasurements': [{'Description': 'The closer '
                                                                   'to 0, the '
                                                                   'smaller '
                                                                   'prediction '
                                                                   'error.',
                                                    'Enable': True,
                                                    'Id': 7,
                                                    'Name': 'RMSE',
                

In [15]:
# generate report
res = helper.post_report(td_1_json['Id'])

<Response [200]>
{'OutParams': {'ReportUrl': 'http://127.0.0.1:8888/qai-testbed/api/0.0.1/download/318'},
 'Result': {'Code': 'D12000', 'Message': 'command invoke success.'}}
